<a href="https://colab.research.google.com/github/bernase/COMP4350-IntroToML/blob/main/ML1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Racing Circuit Curve Analyzer 🏁

*   Melek Berna Şerit 👩🏼‍💻 *18070001027*
*   Doğu Alpay 👨🏻‍💻 *19070001008*

In [18]:
###     Importing Libraries & Loading the data
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [19]:
###     Google Drive connection to access our dataset named DS
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
root_path = '/content/drive/MyDrive/Colab Notebooks/'
sys.path.append(root_path)

Mounted at /content/drive


In [3]:
print("installing casa6 + cngi (takes a minute or two)...")
os.system("apt-get install libgfortran3")
os.system("pip install casatasks==6.3.0.48")
os.system("pip install casadata")
os.system("pip install cngi-prototype==0.0.91")

# Retrieve and extract demonstration datasets
print('retrieving MS tarfiles...')
!gdown -q --id 1N9QSs2Hbhi-BrEHx5PA54WigXt8GGgx1
!tar -xzf sis14_twhya_calibrated_flagged.ms.tar.gz
print('complete')

installing casa6 + cngi (takes a minute or two)...
retrieving MS tarfiles...
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1N9QSs2Hbhi-BrEHx5PA54WigXt8GGgx1 

tar (child): sis14_twhya_calibrated_flagged.ms.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
complete


In [20]:
###     DataSet
dataset_folder_name = os.path.join(root_path, 'DS')   #our dataset folder named DS
MODEL_FILENAME = root_path + "model_cv.h5"
source_files = []
class_labels = ['1', '2', '3', '4', '5', '6']
X = []
Y = []

img_rows, img_cols = 720, 720  # input image dimensions
train_path = os.path.join(dataset_folder_name, 'train')
validation_path = os.path.join(dataset_folder_name, 'validation')
test_path = os.path.join(dataset_folder_name, 'test')

In [5]:
def transfer_between_folders(source, dest, split_rate):
    """ Based on the split ratio this function moves some portion of the source folder to destination folder!

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest locaiton

    """
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(
                dest, str(source_files[each_index])))

    else:
        print("No file moved. Source empty!")

def transfer_all_class_between_folders(source, dest, split_rate):
    """ Transfer the files from source to dest for all the classes. This function calls the 'transfer_between_folders' to actually perform the transfer.

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest locaiton

    """
    for label in class_labels:
        transfer_between_folders(os.path.join(dataset_folder_name, source, label),
                                 os.path.join(
                                     dataset_folder_name, dest, label),
                                 split_rate)


def my_metrics(y_true, y_pred):
    """ Calculate accuracy, precision, and f1 score of the model's prediction with respect to true labels.

        Args:
            y_true: list/array
                All true class labels
            y_pred: list/array
                All predicted class labels

        Returns:
            accuracy: float
                Accuracy measure of the model
            precision: float
                Precision measure of the model
            f1_Score: float
                F1-score measure of the model

    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1_Score = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1_Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1_Score

In [6]:
# check if test folder is empty
transfer_all_class_between_folders('test', 'train', 1.0)

In [7]:
transfer_all_class_between_folders('train', 'test', 0.20)

In [8]:
def prepare_name_with_labels(folder_name, dataset_type='train'):
    """ Prepare the file names (X) and the class labels (Y) from folder location of images.

        Args:
            folder_name: str
                Source folder's path

    """
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = 0
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = i
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [9]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [10]:
X = np.asarray(X)
Y = np.asarray(Y)

In [11]:
batch_size = 10
epoch = 10
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [12]:
def get_model():
    activation_function = 'relu'

    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same',
                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
    model.add(Conv2D(64, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(32, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(16, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(16, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(number_of_class_labels, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


model = get_model()

In [13]:
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train', 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)

    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

Results for fold 1
Found 65 images belonging to 6 classes.
Found 23 images belonging to 6 classes.
Epoch 1/10
7/7 [==============================] - 281s 39s/step - loss: 5.8406 - accuracy: 0.1385
Epoch 2/10
7/7 [==============================] - 285s 40s/step - loss: 1.8244 - accuracy: 0.2000
Epoch 3/10
7/7 [==============================] - 275s 42s/step - loss: 1.7938 - accuracy: 0.1231
Epoch 4/10
7/7 [==============================] - 287s 41s/step - loss: 1.7975 - accuracy: 0.1385
Epoch 5/10
7/7 [==============================] - 275s 39s/step - loss: 1.7954 - accuracy: 0.1077
Epoch 6/10
7/7 [==============================] - 277s 39s/step - loss: 1.7917 - accuracy: 0.1692
Epoch 7/10
7/7 [==============================] - 270s 38s/step - loss: 1.7915 - accuracy: 0.1692
Epoch 8/10
7/7 [==============================] - 280s 43s/step - loss: 1.7913 - accuracy: 0.1692
Epoch 9/10
7/7 [==============================] - 274s 39s/step - loss: 1.7913 - accuracy: 0.1692
Epoch 10/10
3/3 [==

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 65 images belonging to 6 classes.
Found 23 images belonging to 6 classes.
Epoch 1/10
7/7 [==============================] - 275s 38s/step - loss: 1.7911 - accuracy: 0.1538
Epoch 2/10
7/7 [==============================] - 282s 39s/step - loss: 1.7909 - accuracy: 0.1385
Epoch 3/10
7/7 [==============================] - 269s 38s/step - loss: 1.7909 - accuracy: 0.1538
Epoch 4/10
7/7 [==============================] - 278s 42s/step - loss: 1.7905 - accuracy: 0.2000
Epoch 5/10
7/7 [==============================] - 279s 39s/step - loss: 1.7907 - accuracy: 0.2308
Epoch 6/10
7/7 [==============================] - 276s 39s/step - loss: 1.7900 - accuracy: 0.2000
Epoch 7/10
7/7 [==============================] - 273s 38s/step - loss: 1.7912 - accuracy: 0.1077
Epoch 8/10
7/7 [==============================] - 270s 38s/step - loss: 1.7907 - accuracy: 0.1385
Epoch 9/10
7/7 [==============================] - 273s 38s/step - loss: 1.7894 - accuracy: 0.2462
Epoch 10/10
3/3 [=====================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Found 66 images belonging to 6 classes.
Found 22 images belonging to 6 classes.
Epoch 1/10
7/7 [==============================] - 277s 38s/step - loss: 1.7899 - accuracy: 0.2121
Epoch 2/10
7/7 [==============================] - 277s 39s/step - loss: 1.7908 - accuracy: 0.2121
Epoch 3/10
7/7 [==============================] - 280s 39s/step - loss: 1.7907 - accuracy: 0.2121
Epoch 4/10
7/7 [==============================] - 281s 40s/step - loss: 1.7900 - accuracy: 0.1818
Epoch 5/10
7/7 [==============================] - 284s 39s/step - loss: 1.7904 - accuracy: 0.1667
Epoch 6/10
7/7 [==============================] - 277s 39s/step - loss: 1.7908 - accuracy: 0.1818
Epoch 7/10
7/7 [==============================] - 286s 41s/step - loss: 1.7915 - accuracy: 0.1667
Epoch 8/10
7/7 [==============================] - 276s 38s/step - loss: 1.7908 - accuracy: 0.1818
Epoch 9/10
7/7 [==============================] - 286s 40s/step - loss: 1.7895 - accuracy: 0.1667
Epoch 10/10
3/3 [=====================

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False 
)
predictions = model.predict(test_generator, verbose=1)
y_predictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

test_acc, test_prec, test_fScore = my_metrics(true_classes, y_predictions)
model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 12 images belonging to 6 classes.
2/2 [==============================] - 12s 2s/step


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy  : 0.16666666666666666
Precision : 0.027777777777777776
f1Score : 0.047619047619047616
[[0 0 0 0 2 0]
 [0 0 0 0 2 0]
 [0 0 0 0 2 0]
 [0 0 0 0 2 0]
 [0 0 0 0 2 0]
 [0 0 0 0 2 0]]
